In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
image = None
texture = None
file_path = '../../csv/features_data-Sheet1.csv'

In [3]:
# Replace 'your_dataset.csv' with the actual filename of your dataset
df = pd.read_csv(file_path)
#   X = df[['length', 'width', 'diameter', 'area', 'perimeter', 'rectangularity', 'circularity',
#           'ellipticity', 'aspect ratio', 'form factor', 'narrow factor', 'perimeter ratio of diameter',
#           'perimeter Ratio of Physiological Length and Physiological Width', 'Texture']]
#   y = df['Species']

X = df[['circularity','ellipticity', 'form factor', 
        'narrow factor', 'perimeter ratio of diameter',
        'perimeter Ratio of Physiological Length and Physiological Width']]
y = df['Species']

# X = df[['area', 'width', 'circularity', 'ellipticity', 'form factor', 'perimeter',
#         'perimeter ratio of diameter', 'perimeter Ratio of Physiological Length and Physiological Width']]
# y = df['Species']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# FUNCTIONS FOR SVM MACHINE

In [6]:
# Create a dataframe with the dependent and independent variables

def metrics(y_test, y_pred):
    # Evaluate the model on the test set
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)


def train_svm(file_path):
    # Define the SVM classifier
    svm = SVC()

    # Define the hyperparameter grid for tuning
    param_dist = {
        'C': np.logspace(-3, 3, 7),
        'kernel': ['rbf'],
        'gamma': np.logspace(-3, 3, 7)
    }

    # Define the hyperparameter grid for tuning
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['rbf'],
        'degree': [2, 3, 4],
        'gamma': ['scale', 'auto'] + list(np.logspace(-3, 3, 7))
    }

    # Perform Grid Search Cross Validation
    grid_search = GridSearchCV(svm, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    # Print the best hyperparameters and corresponding accuracy
    print("Best Hyperparameters: ", grid_search.best_params_)
    print("Best Accuracy: ", grid_search.best_score_)
    # Train SVM with the best hyperparameters on the entire training set
    best_svm = grid_search.best_estimator_

    # # Create RandomizedSearchCV object
    # random_search = RandomizedSearchCV(estimator=svm, param_distributions=param_dist, n_iter=10, scoring='accuracy', cv=5, random_state=42)
    # random_search.fit(X_train, y_train)
    # # Print the best hyperparameters and corresponding accuracy
    # print("Best Hyperparameters: ", random_search.best_params_)
    # print("Best Accuracy: ", random_search.best_score_ + '\n')
    # # Train SVM with the best hyperparameters on the entire training set
    # best_svm = random_search.best_estimator_

    best_svm.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = best_svm.predict(X_test)

    metrics(y_test, y_pred)
    

    return best_svm

In [7]:
best_svm = train_svm(file_path)

Best Hyperparameters:  {'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Best Accuracy:  0.5399430042287185
Accuracy: 0.4939759036144578
Precision: 0.42096501949828297
Recall: 0.4939759036144578


d:\anaconda3\envs\tf2.4\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
import joblib

# Save the best SVM model
joblib.dump(best_svm, '../../models/plant_prediction_model_svm.pkl')

['../../models/plant_prediction_model_svm.pkl']

In [9]:
import joblib

# Load the saved SVM model
loaded_svm_model = joblib.load('../../models/plant_prediction_model_svm.pkl')

# Make predictions on the test data
y_pred = loaded_svm_model.predict(X_test)

# Evaluate the model on the test set
metrics(y_test, y_pred)

Accuracy: 0.7228915662650602
Precision: 0.7296823658269441
Recall: 0.7228915662650602


In [11]:
# Input new data for prediction
new_data = np.array([[0.6420904344085511,2.040016228764243,1.9064039408866995,0.6420904344085511,0.9853075604106104,2.69546972135897,1.7420684933662414]])  # Replace this with your new data

# Make predictions
predictions = loaded_svm_model.predict(new_data)

# Print the predicted class labels
print("Predicted Class Labels: ", predictions)


Predicted Class Labels:  [0.]


d:\anaconda3\envs\tf2.4\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
